|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 5:</h2>|<h1>Observation (non-causal) mech interp<h1>|
|<h2>Section:</h2>|<h1>Investigating layers<h1>|
|<h2>Lecture:</h2>|<h1><b>CodeChallenge HELPER: Clusters in internal vs. terminal punctuation<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib as mpl

import requests

import torch

# vector plots
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# Exercise 1: Model and punctuation indices

In [ ]:
# load pretrained GPT-2 model and tokenizer
from transformers import AutoModelForCausalLM,GPT2Tokenizer
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

gpt2 = AutoModelForCausalLM.from_pretrained('gpt2-medium')
gpt2 = gpt2.to(device)
gpt2.eval()
nEmb = gpt2.config.n_embd

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')

In [ ]:
text = requests.get('https://www.gutenberg.org/cache/epub/219/pg219.txt').text # Heart of Darkness
tokens =
num_tokens =

print(f'There are {} tokens, {} of which are unique.')

In [ ]:
# tokens to match exactly
internal_punctuations =
terminal_punctuations =


# initialize vector
isPunctuation = np.zeros

# loop over all tokens (ignore starting tokens before the book starts)
for ti in range(

  # current token
  currtok = # also remove preceeding spaces


  # test for punctuation
  if currtok in # if it's an internal punctuation
      isPunctuation[ti] =

  # check if it's terminal -- and not a decimal-point number!
  elif currtok in terminal_punctuations:
    if # not a decimal point
      isPunctuation[ti] = 2


# report
print(f'There are:\n  {sum(isPunctuation==1):3} internal punctuation marks, and\n  {sum(isPunctuation==2):3} terminal punctuation marks.')

In [ ]:
# find the indices
internalIdx =
terminalIdx =

# examine some punctuations
context_win = 9
for t in terminalIdx[:5]:
  print(f'Example {t}:\n{}\n')

# Exercise 2: Create batches and get activations

In [ ]:
# some data parameters
batchsize   =
context_pre =
context_pst =

In [ ]:
# create batches
batch_internal = torch.zeros((batchsize, # initialize to integers
batch_terminal = torch.zeros((batchsize,

# loop over sequences to create batches
for b in range(batchsize):

  # internal punctuations
  tokenLoc = internalIdx[b]
  batch_internal[b,:] =

  # terminal punctuations



batch_terminal.shape

In [ ]:
# process the internal tokens
with torch.no_grad():
  output_internal = gpt2(batch_internal.to(device),output_hidden_states=True)

# repeat for terminal tokens


In [ ]:
print(output_internal.hidden_states[3].shape)
len(output_internal.hidden_states)

In [ ]:
# for convenience, bring the activations to the CPU in a shorter-named variable
hsIntern = []
hsTermin = []

for i in range(len(output_internal.hidden_states)):
  hsIntern.append( output_internal.hidden_states[i] # convenient to have in numpy format
  hsTermin.

In [ ]:
hsIntern[4].shape

In [ ]:
# visualize

# average target tokens over all sequences
ave_intern = hsIntern[3]
ave_termin = hsTermin[3]

_,axs = plt.subplots(1,2,figsize=(10,4))

axs[0].plot(ave_intern,'o',label='Internal')
axs[0].plot(ave_termin,'s',label='Terminal')

axs[1].plot(
axs[1].set(xlabel='Internal',ylabel='Terminal',title='Scatter plot of activations')

plt.tight_layout()
plt.show()

# Exercise 3: MI and cov in one layer

In [ ]:
# a function for mutual information
def mutInfo_and_cov(x,y,outlierThresh=0):

  # remove outliers based on a z-score threshold
  if outlierThresh>0:
    zx = (x-mean) / std
    zy =
    outlier = # if either variable exceeds the threshold
    x = # remove outliers from the data
    y = # make sure the same exact indices are removed from both variables


  # histogram and convert to proportion (estimate of probability)
  Z  = np.histogram2d(x,y,bins=15)[0]
  pZ = # normalize by sum
  px = pZ.sum(axis=1)
  py =

  # calculate entropy
  eps = 1e-12
  Hx = -np.sum( px * np.log2(px+eps) )
  Hy = -np.sum(
  HZ =

  # mutual information
  MI =

  # and covariance
  C = sum( ()*() ) / ()

  return MI,C

In [ ]:
# sanity-check target index ;)
tokenizer.decode(batch_terminal[3,context_pre])

In [ ]:
# this cell takes ~40 sec
whichLayer = 1 # 20

mi_in = np.zeros((batchsize,batchsize))
mi_tr = np.zeros((batchsize,batchsize))
cov_in = np.zeros
cov_tr = np.zeros



# double-loop over the token pairs
for bi in range(batchsize):
  for bj in range(bi+1,batchsize):

    ## internal punctuation
    # extract the data
    x = hsIntern[whichLayer
    y = hsIntern

    # pairwise mutual information and covariance
    mi_in[bi,bj],cov_in[bi,bj] =


    ## repeat for terminal punctuation


In [ ]:
# a variable to extract and vectorize the unique matrix elements
uniqueIndices = np.triu_indices(batchsize
uniqueIndices

In [ ]:
# visualize all pairwise MI from one layer

fig = plt.figure(figsize=(8,6))
gs = GridSpec(2,2,figure=fig)
ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[1,:])


# show the matrices
h = ax0.imshow(,vmin=,vmax=,origin='lower',aspect='auto')
plt.colorbar(h,ax=ax0,fraction=.046,pad=.02)
ax0.set(xticks=[],yticks=[],xlabel='Sequence index',ylabel='Sequence index',title='Pairwise MI for INTERNAL')

h = ax1.imshow(
plt.colorbar(h,ax=ax1,fraction=.046,pad=.02)
ax1.set(xticks=[],yticks=[],xlabel='Sequence index',ylabel='Sequence index',title='Pairwise MI for TERMINAL')


# get histograms from nonzero elements
yIntern,xIntern = np.histogram(mi_in # just the unique elements
yTermin,xTermin = np.histogram(mi_tr

# and show those
ax2.plot(,label='Internal',linewidth=2)
ax2.plot(,label='Terminal',linewidth=2)
ax2.set(xlim=[min(xIntern[0],xTermin[0]),max(xIntern[-1],xTermin[-1])],
        xlabel='Mutual information',ylabel='Count',title='Distribution of MI values')
ax2.legend()


plt.tight_layout()
plt.suptitle(f'Mutual information in layer {whichLayer}',y=1.05,fontweight='bold',fontsize=16)
plt.show()

In [ ]:
# visualize all pairwise covariance from one layer

fig = plt.figure(figsize=(8,6))
gs = GridSpec(2,2,figure=fig)
ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[1,:])


# show the matrices
h = ax0.imshow(
plt.colorbar(h,ax=ax0,fraction=.046,pad=.02)
ax0.set(xticks=[],yticks=[],xlabel='Sequence index',ylabel='Sequence index',title='Pairwise MI for INTERNAL')

h = ax1.imshow(
plt.colorbar(h,ax=ax1,fraction=.046,pad=.02)
ax1.set(xticks=[],yticks=[],xlabel='Sequence index',ylabel='Sequence index',title='Pairwise MI for TERMINAL')


# get histograms from nonzero elements


# and show those

ax2.set(xlim=[min(xIntern[0],xTermin[0]),max(xIntern[-1],xTermin[-1])],
        xlabel='Covariance',ylabel='Log count',title='Distribution of covariance values')
ax2.legend()


plt.tight_layout()
plt.suptitle(f'Covariance in layer {whichLayer}',y=1.05,fontweight='bold',fontsize=16)
plt.show()

In [ ]:
## covariance by mutual information

_,axs = plt.subplots(1,3,figsize=(12,4))

# skip to facilitate plotting
pnts2skip = 17

# plot the internal punctuations
axs[0].plot(,,'.',markerfacecolor=[.7,.7,.9,.3])
axs[0].set(xlabel='Covariance',ylabel='Mutual information',
           title=f'Internal punctuation (r = {np.corrcoef(cov_in[uniqueIndices],mi_in[uniqueIndices])[0,1]:.3f})')

# plot the terminal punctuations

axs[1].set(xlabel='Covariance',ylabel='Mutual information',
           title=f'Terminal punctuation (r = {np.corrcoef(cov_tr[uniqueIndices],mi_tr[uniqueIndices])[0,1]:.3f})')



axs[2].set(xlabel='Covariance',ylabel='Mutual information',title='Both')
axs[2].legend()


plt.tight_layout()
plt.show()

# Exercise 4: MI and cov over all layers

In [ ]:
# initializations
mi_in_all = np.zeros((len(hsIntern),batchsize,batchsize))
mi_tr_all = np.zeros((len(hsIntern),batchsize,batchsize))
cv_in_all = np.zeros((len(hsIntern),batchsize,batchsize))
cv_tr_all = np.zeros((len(hsIntern),batchsize,batchsize))


# loop over layers
for layeri in range(len(hsIntern)):

  # double-loop over the token pairs
  for bi in range(batchsize):
    for bj in range(bi+1,batchsize):

      ## internal punctuation
      x = hsIntern
      y = hsIntern
      mi_in_all[layeri,bi,bj],cv_in_all[layeri,bi,bj] =

      ## repeat for terminal punctuation

  print(f'Finished layer {layeri+1:2}/{len(hsIntern)}')

In [ ]:
# scatter plots
_,axs = plt.subplots(5,5,figsize=(12,11))
axs = axs.flatten()


# skip to facilitate plotting
pnts2skip = 7

for layeri

  # plot the dots
  axs[layeri].plot(cv_in_all[],mi_in_all[],'r.',alpha=.2,label='Intermediate')
  axs[layeri].plot(cv_tr_all[],mi_tr_all[],'k.',alpha=.2,label='Terminal')

  # adjust the axis
  axs[layeri].set(xticks=[],yticks=[],title=f'Layer {layeri}')

axs[20].set(xlabel='Covariance',ylabel='Mutual information')
axs[20].legend(fontsize=8)

plt.tight_layout()
# plt.savefig('ex4.png')
plt.show()

# Exercise 5: Distributions and means

In [ ]:
fig,axs = plt.subplots(2,2,figsize=(10,6))

# normalization function for mapping layer index onto color
norm = mpl.colors.Normalize(vmin=0,vmax=len(hsIntern))

# loop over layers
for layeri in rang

  ### covariance: plot the distribution
  yy,xx = np.histogram(cv_in_all[
  axs[0,0].plot(xx[:-1],yy,color=mpl.cm.plasma(norm(layeri)))

  # and the mean of the distribution
  axs[0,1].plot(layeri,.mean(),'ks',
              markersize=10,markerfacecolor=mpl.cm.plasma(norm(layeri)))



  ### mutual information

  axs[1,0].plot(xx[:-1],yy,color=mpl.cm.plasma(norm(layeri)))

  # and the mean



# add colorbars
sm = mpl.cm.ScalarMappable(cmap=mpl.cm.plasma,norm=norm)
plt.colorbar(sm,ax=axs[0,0],pad=.01)
plt.colorbar(sm,ax=axs[1,0],pad=.01)

# labels and titles
axs[0,0].set(xlabel='Covariance value',ylabel='Density',title='Covariance distribution')
axs[0,1].set(xlabel='Hidden layer',ylabel='Covariance',title='Average covariances')
axs[1,0].set(xlabel='Mutual information value',ylabel='Density',title='MI distribution')
axs[1,1].set(xlabel='Hidden layer',ylabel='Mutual information',title='Average MI values')

plt.tight_layout()
plt.show()